This notebook covers all the optimization steps for training an encoder-decoder transformer model

In [1]:
%autosave 300
%reload_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

Autosaving every 300 seconds


In [2]:
import os

os.chdir(
    "/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers"
)
print(os.getcwd())

/mnt/batch/tasks/shared/LS_root/mounts/clusters/insights-model-run2/code/Users/soutrik.chowdhury/EraV2_Transformers


In [3]:
from S18_code.config import get_config, get_weights_file_path
from S18_code.model import build_transformer
import torch
from S18_code.dataloader import get_ds
from datasets import load_dataset
import torch.optim as optim
import torch.nn as nn
from torchinfo import summary
from torch.optim.lr_scheduler import CyclicLR
from torch.optim import Adam
from torchmetrics.text import BLEUScore
from torch.cuda.amp import GradScaler

In [4]:
device = get_config()["device"]
print(device)

cuda


In [5]:
print(torch.cuda.is_available())
print(torch.cuda.current_device())
print(torch.cuda.get_device_name())
print(torch.cuda.memory_allocated())
print(torch.cuda.memory_reserved())
print(torch.cuda.get_device_name(device.index))
print(torch.cuda.get_device_properties(device.index).total_memory / 1024**3)
print(torch.cuda.memory_summary(device=device.index))
print(torch.cuda.get_device_capability(device.index))
print(torch.cuda.get_device_properties(device.index))
print(torch.cuda.get_device_properties(device.index).multi_processor_count)

True
0
Tesla V100-PCIE-16GB
0
0
Tesla V100-PCIE-16GB
15.7725830078125
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |      0 B   |      0 B   |      0 B   |      0 B   |
|       from large pool |      0 B   |      0 B   |      0 B   |      0 B   |
|       from small pool |      0 B   |      0 B   |      0 B   |      0 B   |
|---------------------------------------------------------------------------|
| Active memory         |      0 B   |      0 B   |      0 B   |      0 

#### Download the dataset and create data via dataloader

In [6]:
ds_raw = load_dataset(
    get_config()["datasource"],
    f"{get_config()['src_lang']}-{get_config()['tgt_lang']}",
    split="train",
)
print(len(ds_raw))

32332


In [7]:
# batch wise padding data loading
train_dataloader, val_dataloader, tokenizer_src, tokenizer_tgt = get_ds(
    ds_raw, get_config()
)

Building tokenizer for en
Tokenizer for en already exists
Building tokenizer for it
Tokenizer for it already exists
Creating the tokenized-dataset
Dataloaders created with 29098 training and 3234 validation samples


In [8]:
for data in train_dataloader:
    print(data["encoder_input"].shape)
    print(data["decoder_input"].shape)
    print(data["encoder_mask"].shape)
    print(data["decoder_mask"].shape)
    break

/anaconda/envs/torch_env/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


torch.Size([16, 70])
torch.Size([16, 44])
torch.Size([16, 1, 1, 70])
torch.Size([16, 1, 44, 44])


In [9]:
for data in val_dataloader:
    print(data["encoder_input"].shape)
    print(data["decoder_input"].shape)
    print(data["encoder_mask"].shape)
    print(data["decoder_mask"].shape)
    break

torch.Size([1, 28])
torch.Size([1, 27])
torch.Size([1, 1, 1, 28])
torch.Size([1, 1, 27, 27])


#### Model Initialization and Training

In [10]:
transformer_model = build_transformer(
    src_vocab_size=tokenizer_src.get_vocab_size(),
    tgt_vocab_size=tokenizer_tgt.get_vocab_size(),
).to(device)

In [11]:
summary(transformer_model)

Layer (type:depth-idx)                                  Param #
TransformerModel                                        --
├─Encoder: 1-1                                          --
│    └─ModuleList: 2-1                                  --
│    │    └─EncoderBlock: 3-1                           3,150,336
│    │    └─EncoderBlock: 3-2                           3,150,336
│    │    └─EncoderBlock: 3-3                           3,150,336
│    │    └─EncoderBlock: 3-4                           3,150,336
│    │    └─EncoderBlock: 3-5                           3,150,336
│    │    └─EncoderBlock: 3-6                           3,150,336
│    └─LayerNormalization: 2-2                          1,024
├─Decoder: 1-2                                          --
│    └─ModuleList: 2-3                                  --
│    │    └─DecoderBlock: 3-7                           4,199,936
│    │    └─DecoderBlock: 3-8                           4,199,936
│    │    └─DecoderBlock: 3-9                      

First we will do basic training and validation based on:
* Application of basic training loop
* Application of custom scheduler for learning rate

In [12]:
from S18_code.trainer import (
    CustomLRScheduler,
    TranslationLoss,
    run_training_loop_basic,
    run_validation_loop,
    greedy_decode,
    run_inference_loop,
    run_training_loop_opt,
    run_training_loop_opt_clip,
)

from S18_code.utils import start_timer, end_timer

In [13]:
pad_idx = tokenizer_src.token_to_id("[PAD]")
sos_idx = tokenizer_src.token_to_id("[SOS]")
eos_idx = tokenizer_src.token_to_id("[EOS]")
label_smoothing = 0.1

In [14]:
# Optimizer Adam with weight decay
optimizer = Adam(
    transformer_model.parameters(),
    betas=(0.9, 0.98),
    eps=1.0e-9,
    lr=get_config()["learning_rate"],
)
# Learning rate scheduler
scheduler = CustomLRScheduler(optimizer, get_config()["d_model"], 1000)
# Loss function
criterion = TranslationLoss(pad_idx, label_smoothing, tokenizer_tgt)
# BLEU score metric
metric = BLEUScore()

/anaconda/envs/torch_env/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Basic Training

In [15]:
# for epoch in range(get_config()["num_epochs"]):
#     global_step = 0
#     train_loss, global_step = run_training_loop_basic(
#         transformer_model,
#         train_dataloader,
#         optimizer,
#         criterion,
#         device,
#         global_step,
#         scheduler,
#     )
#     print(f"Epoch: {epoch}, Train loss: {train_loss}")

#     val_loss = run_validation_loop(transformer_model, val_dataloader, criterion, device)
#     print(f"Epoch: {epoch}, Validation loss: {val_loss}")

#     run_inference_loop(
#         transformer_model,
#         val_dataloader,
#         tokenizer_tgt,
#         device,
#         5,
#         metric,
#         sos_idx,
#         eos_idx,
#     )

Application of optimization techniques on training loop to improve performance:
* Automatic Mixed Precision
* Gradient Scaling (Gradient Clipping)

In [16]:
scaler = torch.cuda.amp.GradScaler()

In [17]:
start_timer()

for epoch in range(get_config()["num_epochs"]):
    global_step = 0
    train_loss, global_step = run_training_loop_opt_clip(
        transformer_model,
        train_dataloader,
        optimizer,
        criterion,
        device,
        global_step,
        scaler,
        scheduler,
    )
    print(f"Epoch: {epoch}, Train loss: {train_loss}")

    val_loss = run_validation_loop(transformer_model, val_dataloader, criterion, device)
    print(f"Epoch: {epoch}, Validation loss: {val_loss}")

    run_inference_loop(
        transformer_model,
        val_dataloader,
        tokenizer_tgt,
        device,
        5,
        metric,
        sos_idx,
        eos_idx,
    )

end_timer("Simple Neural Network")

/anaconda/envs/torch_env/lib/python3.10/site-packages/torch/cuda/memory.py:330: FutureWarning: torch.cuda.reset_max_memory_allocated now calls torch.cuda.reset_peak_memory_stats, which resets /all/ peak memory stats.
  warnings.warn(


Max memory used in start: 0.28GB


Training:   0%|                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               | 0/1819 [00:00<?, ?it/s]/

Epoch: 0, Train loss: 6.395487659248826


Validation: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3234/3234 [01:32<00:00, 35.05it/s, loss=6.284]


...

Inference:   0%|█▏                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 4/3234 [00:02<39:46,  1.35it/s]

SOURCE: Levin had been five days in Kashin, going daily to meetings and taking a great deal of trouble over his sister's business, which he was still unable to arrange.
TARGET: Levin era a Kašin già da sei giorni, frequentando tutti i giorni l’assemblea e affannandosi per l’affare della sorella che non riusciva a sistemare.
PREDICTED: ma non voleva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire , ma non poteva dire , ma non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva


Inference:   0%|█▍                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    | 5/3234 [00:04<45:19,  1.19it/s]


SOURCE: Fancies, memories, and most strange thoughts followed one another with extreme rapidity and clearness: now he saw himself pouring out medicine for the patient and overfilling the spoon, then he saw the midwife's white hands, or Karenin's curious pose as he knelt on the floor by her bedside.
TARGET: Le immagini, le memorie e le idee più strane si susseguivano le une alle altre con straordinaria velocità e chiarezza: ora la medicina che aveva versato all’ammalata e che aveva fatto gocciolare dal cucchiaino, ora le braccia bianche della levatrice, o la strana posizione di Aleksej Aleksandrovic sul pavimento, davanti al letto.
PREDICTED: ma non voleva dire che non poteva dire che , ma non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire , ma non poteva dire , ma non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che non poteva dire che 

Training:  70%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                                                                                                                                                                                                                         | 1272/1819 [03:09<01:19,  6.91it/s, loss=5.810]

In [ ]:
# Next 10

In [ ]:
# start_timer()

# for epoch in range(get_config()["num_epochs"]):
#     global_step = 0
#     train_loss, global_step = run_training_loop_opt_clip(
#         transformer_model,
#         train_dataloader,
#         optimizer,
#         criterion,
#         device,
#         global_step,
#         scaler,
#         scheduler,
#     )
#     print(f"Epoch: {epoch}, Train loss: {train_loss}")

#     val_loss = run_validation_loop(transformer_model, val_dataloader, criterion, device)
#     print(f"Epoch: {epoch}, Validation loss: {val_loss}")

#     run_inference_loop(
#         transformer_model,
#         val_dataloader,
#         tokenizer_tgt,
#         device,
#         5,
#         metric,
#         sos_idx,
#         eos_idx,
#     )

# end_timer("Simple Neural Network")

the slow update of the model is due to LR scheduler. The LR scheduler is not updating the LR as expected